# Mount Drive


In [0]:
from os.path import join
from google.colab import drive
 
ROOT = "/content/drive"
drive.mount(ROOT)
 
PROJ = "My Drive/DetectConcealGun" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
 

%cd ~/content
%cd drive/My Drive/DetectConcealGun/app

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[Errno 2] No such file or directory: '/root/content'
/content
/content/drive/My Drive/DetectConcealGun/app


# Detecting Concealed Gun

In [0]:
!pip install flask-ngrok

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import render_template, request, redirect, flash, url_for
from werkzeug.utils import secure_filename
import os
import subprocess
import json
import pandas as pd
import numpy as np
import glob
import cv2
import pickle
from sklearn.preprocessing import StandardScaler
import shutil
import pathlib
from statistics import mean 


app = Flask(__name__, static_folder="files")
run_with_ngrok(app)   #starts ngrok when the app is run

app.config["VIDEO_UPLOADS"] = "/content/drive/My Drive/DetectConcealGun/app/files"
app.config["ALLOWED_VIDEO_EXTENSIONS"] = ["MP4","MOV","AVI"]


def convert_tomp4(clipname):
  clipnames = clipname.split(".")
  avi_file_path = '/content/drive/My Drive/DetectConcealGun/app/files/result.avi'
  output_name = 'app/files/results/result_' + clipnames[0]
  print(output_name)
  os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
  os.chdir("/content/drive/My Drive/DetectConcealGun/app")
  print("It's success!")

def framestoVDO(clipname):
  os.chdir("/content/drive/My Drive/DetectConcealGun")
  path = 'app/files/images/'
  img_array = []
  for filename3 in glob.glob(path+'*.jpg'):
    img = cv2.imread(filename3)
    height, width, layers = img.shape
    sizes = (width,height)
    img_array.append(img) 
  
  out = cv2.VideoWriter('app/files/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, sizes)
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()
  print("video created")
  convert_tomp4(clipname)



#NOTIFY LINE*****

def notifyFile(filename):
    file = {'imageFile':open(filename,'rb')}
    payload = {'message': 'พบบุคคลที่มีความเสี่ยงในการอำพรางอาวุธปืนในพื้นที่ของคุณ!'}
    return _lineNotify(payload,file)

def _lineNotify(payload,file=None):
    import requests
    url = 'https://notify-api.line.me/api/notify'
    token = '7JODPqwRqL73hrp1To1AYma0bI7evyKTcHtxXnjulWX'	#EDIT
    headers = {'Authorization':'Bearer '+token}
    return requests.post(url, headers=headers , data = payload, files=file)   

def Notify_Line(APIC):
  os.chdir("/content/drive/My Drive/DetectConcealGun")
  aray1 = list()
  for f in glob.glob('app/images/*.jpg'):
      aa1 = (os.path.split(f)[-1])
      aray1.append(aa1)
  piclist = list()
  meanpic = list()
  for i in aray1 :
    person = APIC[APIC[0]==i]
    per = person[person['Class'] == 1]
    if len(per) > 0:
      bb = i
      piclist.append(bb)
    piclist_df = pd.DataFrame(piclist)

  if len(piclist) != 0:
    for n in piclist : ##
      person = APIC[APIC[0]==n]

      r =  [1.0,2.0,3.0,4.0,5.0]
      arealist = list()
      #Pose position person /
      for m in r :
        per = person[person[52] == m]
        if len(per) > 0:
          #select body point
          X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
          X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
          Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
          Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
          area = (Y_max-Y_min)*(X_max-X_min) #Box area 
          arealist.append(int(area))
          meanlist = mean(arealist)
      meanpic.append(int(meanlist))
      meanpic_df = pd.DataFrame(meanpic)
    meanpic_df.reset_index(inplace=True) #set key
    piclist_df.reset_index(inplace=True)#set key
    meanpicdf = piclist_df.join(meanpic_df, on='index', how='left', rsuffix='mean')
    meanpicdf = meanpicdf.drop(columns=['index','indexmean'])
    Maxarea = meanpicdf.max()
  if len(piclist) != 0:
    print(notifyFile('app/files/images/'+Maxarea[0]))


def person_box(APIC,video_input):
  APIC3 = APIC
  aray = list()
  for f in glob.glob('image/*.jpg'):
      aa = (os.path.split(f)[-1])
      aray.append(aa)

  if os.path.exists('app/files/images') :
    shutil.rmtree('app/files/images')
  os.mkdir('app/files/images')
  path = 'app/files/image/'
  for i in aray : ##
      images = cv2.imread(path+i,cv2.IMREAD_COLOR)

      person = APIC[APIC[0]==i]
      r =  [1.0,2.0,3.0,4.0,5.0]
      #Pose position person /
      for m in r :
          per = person[person[52] == m]
          if len(per) > 0:
              #select body point
              X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
              X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
              Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
              Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
              clas = int(per['Class']) #
              if clas == 1:
                  color = (0,0,255) #RED
                  thickness = 20
                  images = cv2.rectangle(images, (X_max,Y_min), (X_min,Y_max), color, thickness)
                  scor = float("%.4f"% per['Class_1'])*100
                  name = 'Gun'+': '+str(scor)+'%'
                  font = cv2.FONT_HERSHEY_SIMPLEX 
                  fontScale = 2
                  # Line thickness of 2 px 
                  thicknes = 7
                  images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA) 
              else :
                  color = (0, 255, 0) #GREEN
                  thickness = 20
                  images = cv2.rectangle(images,(X_max,Y_min), (X_min,Y_max), color, thickness)
                  scor = float("%.4f"% per['Class_0'])*100
                  name = 'None'+': '+str(scor)+'%'
                  font = cv2.FONT_HERSHEY_SIMPLEX
                  fontScale = 2
                  # Line thickness of 2 px 
                  thicknes = 7
                  images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA)
              cv2.imwrite('app/files/images/'+i, images)
  Notify_Line(APIC)
  framestoVDO(video_input)






def data_join (file1, file2,video_input):
  #load Class
  Class = pd.read_csv(file1) #2041(Class), 2042(Scores)
  print('CLASS TABLE','==========>','\n',Class)
  #load APIC
  APIC = pd.read_csv(file2,header=None) 
  print('APIC TABLE','==========>','\n',APIC)
  r =  [1.0,2.0,3.0,4.0,5.0]
  m = range(0,5)
  A = APIC[52]
  B = APIC[52]#*Sores0
  C = APIC[52]#*Sores1
  for n in m :
    di = Class[Class['Unnamed: 0']==n]
    if len(di) > 0:
      clas = int(di['0_Predict'])
      cl = int(di['Unnamed: 0'])
      score_0 = float(di['Class_0'])
      score_1 = float(di['Class_1'])
      if cl == 0:
        A = A.replace(1.0,clas)
        B = B.replace(1.0,score_0)
        C = C.replace(1.0,score_1)
      elif cl == 1:
        A = A.replace(2.0,clas)
        B = B.replace(2.0,score_0)
        C = C.replace(2.0,score_1)
      elif cl == 2:
        A = A.replace(3.0,clas)
        B = B.replace(3.0,score_0)
        C = C.replace(3.0,score_1)
      elif cl == 3:
        A = A.replace(4.0,clas)
        B = B.replace(4.0,score_0)
        C = C.replace(4.0,score_1)
      elif cl == 4:
        A = A.replace(5.0,clas)
        B = B.replace(5.0,score_0)
        C = C.replace(5.0,score_1)

  APIC['Class'] = A
  APIC['Class_0'] = B
  APIC['Class_1'] = C
  #check person
  person = len(Class)
  APIC = APIC[APIC[52] <= person]
  person_box(APIC,video_input)

def mode_RF(Data,video_input): 
  #reset index
  Data.reset_index(drop=True, inplace=True) 
  X_test = Data.values.tolist()#dataframe to list
  
  #Load model
  filename = "app/model_RF.pkl"
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  
  #prediction
  y_pred= model.predict(X_test)

  # show the inputs and predicted outputs
  P_data=list()
  for i in range(len(X_test)):
    P_out = y_pred[i]
    P_data.append(P_out) 
  # Calling DataFrame constructor on list 
  P_datadf = pd.DataFrame(P_data) 
  # check class
  Data.reset_index(drop=True, inplace=True)#reset index
  Data.reset_index(inplace=True) #set key
  P_datadf.reset_index(inplace=True)#set key
  # keep the predictions for class 
  df = Data.join(P_datadf, on='index', how='left', rsuffix='_Predict') #join left
  df = df.drop(columns=['index_Predict']) #drop don't select

    
  #prop each class
  prob = model.predict_proba(X_test)
  prob_out_df = list()
  for i in range(len(X_test)):
    prob_out = prob[i]
    prob_out_df.append(prob_out) 
  # Calling DataFrame constructor on list 
  prob_df = pd.DataFrame(prob_out_df) 
  prob_df.reset_index(drop=True, inplace=True)#reset index
  prob_df.reset_index(inplace=True) #set key
  prob_df = prob_df.rename(columns={0: "Class_0", 1: "Class_1"}) #ChangName
  df = df.join(prob_df, on='index', how='left', rsuffix='_')
  df = df.drop(columns=['index','index_'])

  # threshold < 0.6 
  m = range(0,5)
  for n in m :
    di = df[df['index']==n]
    cla = di['Class_1'] 
    for k in cla :
      if k < 0.6 :
        di['0_Predict'].replace({1: 0}, inplace=True)
    df[df['index']==n] = di
  df = df.drop(columns=['index'])
  print('Check Class for Prediction***', '\n', df)
  # Save .CSV
  df.to_csv('result_model_prediction.csv')
  data_join('app/files/result_model_prediction.csv', 'app/files/save_apic.csv',video_input)

def data_train(video_input):
  file= 'app/files/Fature_to_Test.csv'
  Data = pd.read_csv(file,header=None)
  mode_RF(Data,video_input)

def prepare_idx(idkp,video_input):
  maxjpg =[]
  for key, value in idkp.items():
    maxjpg.append(max([int(s) for s in key.split('.') if s.isdigit()]))
  maxjpg = max(maxjpg)
  print('max.jpg = ',maxjpg,'.jpg')

  name_column=['XNose','YNose','XLEye','YLEye','XREye','YREye','XLEar','YLEar','XREar','YREar','XLShoulder','YLShoulder','XRShoulder','YRShoulder','XLElbow','YLElbow','XRElbow','YRElbow','XLWrist','YLWrist','XEWrist','YEWrist','XLHip','YLHip','XRHip','YRHip','XLKnee','YLKnee','XRKnee','YRKnee','XLAnkle','YLAnkle','XRAnkle','YRAnkle']
  featvec = list()
  for idxn in range(1,6,1) :
    # create list name pic .jpg ให้วนลูป
    jpglist  = list()
    pic = ['%s.jpg']
    for f in range(1,maxjpg+1,1): 
      link = pic[0] % (f)
      jpglist.append(link)

    frame, keypoints, idx = [], [] , []
    for key, value in idkp.items():
      picx = idkp[(key)]
      for p in picx:
        frame.append(key)
        keypoints.append(p['keypoints'])
        idx.append(p['idx'])
        df = pd.DataFrame() # create empty dataframe 
        df['frame'] = frame
        df['keypoints'] = keypoints
        df['idx'] = idx
        apic = df
    
    jpglistdf = pd.DataFrame(jpglist, columns=['frame']) # Create jpglist dataframe
    pointn = apic[apic['idx'] == (idxn)]
    allFrame = pd.merge(jpglistdf , pointn[['frame', 'keypoints']], on='frame', how='left')
    # Replace NaN with list [-1,-1,-1, .  .  . ,-1] (51 item totol)
    allFrame.loc[allFrame['keypoints'].isnull(),['keypoints']] = allFrame.loc[allFrame['keypoints'].isnull(),'keypoints'].apply(lambda keypoints: [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
    # Keypoint
    kpall = allFrame['keypoints']
    # remove c 
    keypointls = list()
    for frm in kpall:
      frm = [round(i,3) for i in frm]  # convert a list into list with 3 decimal places
      for i in frm:
        if i <= 1 and i >= 0 :
            frm.remove(i)
      keypointls.append(frm)

    # convert list to array
    keypointar = np.asarray(keypointls)

    #ปรับค่า X,Y Nose
    arrayper = keypointar
    X_Zero = arrayper[:,0] #X point Zero
    Y_Zero = arrayper[:,1] #Y point Zero
    l=list()
    for i in range(0, 34):
      if(i%2) == 0:
        A = abs(arrayper[:,i] - X_Zero)
      else:
        A = abs(arrayper[:,i] - Y_Zero)
      l.append(A)

    #Z Score
    
    scaler = StandardScaler()
    scaler.fit(l)
    lstand = scaler.transform(l)
    datalistdf = pd.DataFrame(lstand, name_column)
    df = datalistdf.T

    #กรณี มากกว่า 60 เฟรม จะเริ่มเก็บจากเฟรมสุดท้าย
    (naa,nbb) = lstand.shape
    D1 = list()
    for r in range(0,34):
      for t in range(nbb-60,nbb):
        dat1 = lstand[r,t]
        D1.append(dat1)
    datalistdf1 = pd.DataFrame(D1)
    df1 = datalistdf1.T
    df2 = df.values.tolist()

    # featvec.append(df1)
    # กรณีที่จับได้น้อยกว่า 50%
    ccv0 = 0
    for cv1 in df2:
      for cv0 in cv1:
        if cv0 == 0:
          ccv0 = ccv0 + 1
    if ccv0 < 1700:  #old ccv0 < 1020
      featvec.append(df1)

  featvec2 = pd.concat(featvec)
  np.savetxt('app/files/Fature_to_Test.csv', featvec2,delimiter=',' )
  np.savetxt('app/files/save_apic.csv',apic,fmt=['%s','%s', '%f'],delimiter=',')
  data_train(video_input)

def read_json(video_input): #read json file
  os.chdir('/content/drive/My Drive/DetectConcealGun')
  with open('app/files/alphapose-results-forvis-tracked.json','r') as myfile:
    data = myfile.read()
    obj = json.loads(data)
    prepare_idx(obj,video_input)


def  get_poseflow(video_input):
  if os.path.exists('image') :
    shutil.rmtree('image')
  os.mkdir("image")
  !ffmpeg  -i $video_input -r 30 image/%d.jpg    #Extract images

  if os.path.exists('Outimages') : shutil.rmtree('Outimages')
  os.mkdir ("Outimages")
  !pip install -q youtube-dl visdom
  os.chdir( "AlphaPose" )
  !python3 demo.py --sp --indir ../image/ --outdir ../.
  !pip2 install munkres==1.0.12   #poseflow
  os.chdir('/content/drive/My Drive/DetectConcealGun/app/files')
  !python2  AlphaPose/PoseFlow/tracker-general.py --imgdir image/ --in_json alphapose-results.json --out_json alphapose-results-forvis-tracked.json --visdir Outimages/

  os.chdir('/content/drive/My Drive/DetectConcealGun')
  read_json(video_input)

def get_length(filename):
  result = subprocess.run(["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  print('Video length : ' ,float(result.stdout))
  if float(result.stdout) < 2:
    print ("Please upload your video less than 2 second")
  elif float(result.stdout) > 100 :
    print ("Sorry this video is more than 2 second video")
  else :
    get_poseflow(filename)

def main(videoName):
  os.chdir("/content/drive/My Drive/DetectConcealGun/app/files")
  # videoName = str(input("Enter Video name : "))
  get_length(videoName)
  # os.chdir("/content/drive/My Drive/DetectConcealGun")



def allowed_video(filename):
    if not "." in filename:
        return False
    ext = filename.rsplit(".", 1)[1]
    if ext.upper() in app.config["ALLOWED_VIDEO_EXTENSIONS"]:
        return True
    else:
        return False

@app.route("/", methods=["GET", "POST"])
def upload_video():
    if request.method == "POST":
        if request.files:
            video = request.files["video"]

            videoin = 'files/' + video.filename

            if os.path.exists(videoin) :
              print('This is the same video')
              filename = secure_filename(video.filename)
              # read_json(filename)
              filenameout = 'results/result_' +secure_filename(video.filename)
              return render_template("public/result_video.html", videoresult=filenameout)

            if video.filename == "":
                # flash("No file", "warning")
                return redirect(request.url)


            if allowed_video(video.filename):
                filename = secure_filename(video.filename)
                video.save(os.path.join(app.config["VIDEO_UPLOADS"], filename))
                # flash("Video uploaded", "success")


                # main(filename)
                read_json(filename)
                # framestoVDO(filename)
                
                # return redirect(request.url)
                filenameout = 'results/result_' + secure_filename(video.filename)
                return render_template("public/result_video.html", videoresult=filenameout)
                # return redirect(url_for("result_detect",filename=filename))

            else:
                # flash("That file extension is not allowed", "danger")
                return redirect(request.url)
    return render_template("public/upload_video.html")


#PAGE

@app.route("/about")
def about():
    return render_template("public/about.html")
  
@app.route("/result_video")
def result_video():
  return render_template("public/result_video.html")

@app.route("/dataset")
def video_dataset():
  return render_template("public/dataset.html")


app.run()

In [0]:
newjson = video_input.split(".")
dst_dir= '/content/drive/My Drive/DetectConcealGun/app/files/result'
src_file = '/content/drive/My Drive/DetectConcealGun/app/files/alphapose-results-forvis-tracked.json'
shutil.copy(src_file,dst_dir)

dst_file = '/content/drive/My Drive/DetectConcealGun/app/files/result/alphapose-results-forvis-tracked.json'
new_dst_file_name = dst_dir + 'tracked_' + newjson[0] +'.json'
os.rename(dst_file, new_dst_file_name)

# Install flask

In [0]:
!pip install flask-ngrok

# New Section 2


In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import render_template, request, redirect, flash, url_for
from werkzeug.utils import secure_filename
import os
import subprocess
import json
import pandas as pd
import numpy as np
import glob
import cv2
import pickle
from sklearn.preprocessing import StandardScaler
import shutil
import pathlib
from statistics import mean 


app = Flask(__name__, static_folder="files")
run_with_ngrok(app)   #starts ngrok when the app is run

app.config["VIDEO_UPLOADS"] = "/content/drive/My Drive/DetectConcealGun/app/files"
app.config["ALLOWED_VIDEO_EXTENSIONS"] = ["MP4","MOV","AVI"]


def convert_tomp4(clipname):
  clipnames = clipname.split(".")
  avi_file_path = '/content/drive/My Drive/DetectConcealGun/app/files/result.avi'
  output_name = 'app/files/results/result_' + clipnames[0]
  print(output_name)
  os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
  print("It's success!")

def framestoVDO(clipname):
  os.chdir("/content/drive/My Drive/DetectConcealGun")
  path = 'app/files/images/'
  img_array = []
  for filename3 in glob.glob(path+'*.jpg'):
    img = cv2.imread(filename3)
    height, width, layers = img.shape
    sizes = (width,height)
    img_array.append(img) 
  
  out = cv2.VideoWriter('app/files/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, sizes)
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()
  print("video created")
  convert_tomp4(clipname)


#NOTIFY LINE*****

def notifyFile(filename2,video_input):
    file = {'imageFile':open(filename2,'rb')}
    payload = {'message': 'พบบุคคลที่มีความเสี่ยงในการอำพรางอาวุธปืนในพื้นที่ของคุณ!'}
    return _lineNotify(payload,file)

def _lineNotify(payload,file=None):
    import requests
    url = 'https://notify-api.line.me/api/notify'
    token = '7JODPqwRqL73hrp1To1AYma0bI7evyKTcHtxXnjulWX'	#EDIT
    headers = {'Authorization':'Bearer '+token}
    return requests.post(url, headers=headers , data = payload, files=file)
      
def Notify_Line(APIC,video_input):
  os.chdir("/content/drive/My Drive/DetectConcealGun/app/files")
  aray1 = list()
  for f in glob.glob('images/*.jpg'):
      aa1 = (os.path.split(f)[-1])
      aray1.append(aa1)
  piclist = list()
  meanpic = list()
  for i in aray1 :
    person = APIC[APIC[0]==i]
    per = person[person['Class'] == 1]
    if len(per) > 0:
      bb = i
      piclist.append(bb)
    piclist_df = pd.DataFrame(piclist)

  if len(piclist) != 0:
    for n in piclist : ##
      person = APIC[APIC[0]==n]

      r =  [1.0,2.0,3.0,4.0,5.0]
      arealist = list()
      #Pose position person /
      for m in r :
        per = person[person[52] == m]
        if len(per) > 0:
          #select body point
          X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
          X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
          Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
          Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
          area = (Y_max-Y_min)*(X_max-X_min) #Box area 
          arealist.append(int(area))
          meanlist = mean(arealist)
      meanpic.append(int(meanlist))
      meanpic_df = pd.DataFrame(meanpic)
    meanpic_df.reset_index(inplace=True) #set key
    piclist_df.reset_index(inplace=True)#set key
    meanpicdf = piclist_df.join(meanpic_df, on='index', how='left', rsuffix='mean')
    meanpicdf = meanpicdf.drop(columns=['index','indexmean'])
    Maxarea = meanpicdf.max()
  if len(piclist) != 0:
    nnoti = 'images/'+ Maxarea[0]
    notifyFile(nnoti,video_input)


def person_box(APIC,video_input):
  APIC3 = APIC
  aray = list()
  for f in glob.glob('app/files/image/*.jpg'):
      aa = (os.path.split(f)[-1])
      aray.append(aa)

  if os.path.exists('app/files/images') :
    shutil.rmtree('app/files/images')
  os.mkdir('app/files/images')
  path = 'app/files/image/'
  for i in aray : ##
      images = cv2.imread(path+i,cv2.IMREAD_COLOR)

      person = APIC[APIC[0]==i]
      r =  [1.0,2.0,3.0,4.0,5.0]
      #Pose position person /
      for m in r :
          per = person[person[52] == m]
          if len(per) > 0:
              #select body point
              X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
              X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
              Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
              Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
              clas = int(per['Class']) #
              if clas == 1:
                  color = (0,0,255) #RED
                  thickness = 20
                  images = cv2.rectangle(images, (X_max,Y_min), (X_min,Y_max), color, thickness)
                  scor = float("%.4f"% per['Class_1'])*100
                  scor = float("%.4f"% scor)
                  name = 'Gun'+': '+str(scor)+'%'
                  font = cv2.FONT_HERSHEY_SIMPLEX 
                  fontScale = 2
                  # Line thickness of 2 px 
                  thicknes = 7
                  images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA) 
              else :
                  color = (0, 255, 0) #GREEN
                  thickness = 20
                  images = cv2.rectangle(images,(X_max,Y_min), (X_min,Y_max), color, thickness)
                  scor = float("%.4f"% per['Class_0'])*100
                  scor = float("%.4f"% scor)
                  name = 'None'+': '+str(scor)+'%'
                  font = cv2.FONT_HERSHEY_SIMPLEX
                  fontScale = 2
                  # Line thickness of 2 px 
                  thicknes = 7
                  images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA)
              cv2.imwrite('app/files/images/'+i, images)
  Notify_Line(APIC3,video_input)
  framestoVDO(video_input) 


def data_join (file1, file2,video_input):
  #load Class
  Class = pd.read_csv(file1) #2041(Class), 2042(Scores)
  #load APIC
  APIC = pd.read_csv(file2,header=None) 
  r =  [1.0,2.0,3.0,4.0,5.0]
  m = range(0,5)
  A = APIC[52]
  B = APIC[52]#*Sores0
  C = APIC[52]#*Sores1
  for n in m :
    di = Class[Class['Unnamed: 0']==n]
    if len(di) > 0:
      clas = int(di['0_Predict'])
      cl = int(di['Unnamed: 0'])
      score_0 = float(di['Class_0'])
      score_1 = float(di['Class_1'])
      if cl == 0:
        A = A.replace(1.0,clas)
        B = B.replace(1.0,score_0)
        C = C.replace(1.0,score_1)
      elif cl == 1:
        A = A.replace(2.0,clas)
        B = B.replace(2.0,score_0)
        C = C.replace(2.0,score_1)
      elif cl == 2:
        A = A.replace(3.0,clas)
        B = B.replace(3.0,score_0)
        C = C.replace(3.0,score_1)
      elif cl == 3:
        A = A.replace(4.0,clas)
        B = B.replace(4.0,score_0)
        C = C.replace(4.0,score_1)
      elif cl == 4:
        A = A.replace(5.0,clas)
        B = B.replace(5.0,score_0)
        C = C.replace(5.0,score_1)

  APIC['Class'] = A
  APIC['Class_0'] = B
  APIC['Class_1'] = C
  #check person
  person = len(Class)
  APICto = APIC[APIC[52] <= person]
  person_box(APICto,video_input)

def mode_RF(Data,video_input): 
  #reset index
  Data.reset_index(drop=True, inplace=True) 
  X_test = Data.values.tolist()#dataframe to list
  
  #Load model
  filename = "app/model_RF.pkl"
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  
  #prediction
  y_pred= model.predict(X_test)

  # show the inputs and predicted outputs
  P_data=list()
  for i in range(len(X_test)):
    P_out = y_pred[i]
    P_data.append(P_out) 
  # Calling DataFrame constructor on list 
  P_datadf = pd.DataFrame(P_data) 
  # check class
  Data.reset_index(drop=True, inplace=True)#reset index
  Data.reset_index(inplace=True) #set key
  P_datadf.reset_index(inplace=True)#set key
  # keep the predictions for class 
  df = Data.join(P_datadf, on='index', how='left', rsuffix='_Predict') #join left
  df = df.drop(columns=['index_Predict']) #drop don't select

    
  #prop each class
  prob = model.predict_proba(X_test)
  prob_out_df = list()
  for i in range(len(X_test)):
    prob_out = prob[i]
    prob_out_df.append(prob_out) 
  # Calling DataFrame constructor on list 
  prob_df = pd.DataFrame(prob_out_df) 
  prob_df.reset_index(drop=True, inplace=True)#reset index
  prob_df.reset_index(inplace=True) #set key
  prob_df = prob_df.rename(columns={0: "Class_0", 1: "Class_1"}) #ChangName
  df = df.join(prob_df, on='index', how='left', rsuffix='_')
  df = df.drop(columns=['index_'])

  # threshold < 0.6 

  m = range(0,5)
  for n in m :
    di = df[df['index']==n]
    cla = di['Class_1'] 
    for k in cla :
      if k < 0.6 :
        di['0_Predict'].replace({1: 0}, inplace=True)
    df[df['index']==n] = di
  df = df.drop(columns=['index'])
  print('Check Class for Prediction***', '\n', df)

  #Save .CSV
  df.to_csv('app/files/result_model_prediction.csv') #save result
  data_join ('app/files/result_model_prediction.csv', 'app/files/save_apic.csv',video_input)

def data_train(video_input):
  file= 'app/files/Fature_to_Test.csv'
  Data = pd.read_csv(file,header=None)
  mode_RF(Data,video_input)

def prepare_idx(idkp,video_input):
  maxjpg =[]
  for key, value in idkp.items():
    maxjpg.append(max([int(s) for s in key.split('.') if s.isdigit()]))
  maxjpg = max(maxjpg)
  print('max.jpg = ',maxjpg,'.jpg')

  name_column=['XNose','YNose','XLEye','YLEye','XREye','YREye','XLEar','YLEar','XREar','YREar','XLShoulder','YLShoulder','XRShoulder','YRShoulder','XLElbow','YLElbow','XRElbow','YRElbow','XLWrist','YLWrist','XEWrist','YEWrist','XLHip','YLHip','XRHip','YRHip','XLKnee','YLKnee','XRKnee','YRKnee','XLAnkle','YLAnkle','XRAnkle','YRAnkle']
  featvec = list()
  for idxn in range(1,6,1) :
    # create list name pic .jpg ให้วนลูป
    jpglist  = list()
    pic = ['%s.jpg']
    for f in range(1,maxjpg+1,1): 
      link = pic[0] % (f)
      jpglist.append(link)

    frame, keypoints, idx = [], [] , []
    for key, value in idkp.items():
      picx = idkp[(key)]
      for p in picx:
        frame.append(key)
        keypoints.append(p['keypoints'])
        idx.append(p['idx'])
        df = pd.DataFrame() # create empty dataframe 
        df['frame'] = frame
        df['keypoints'] = keypoints
        df['idx'] = idx
        apic = df
    
    jpglistdf = pd.DataFrame(jpglist, columns=['frame']) # Create jpglist dataframe
    pointn = apic[apic['idx'] == (idxn)]
    allFrame = pd.merge(jpglistdf , pointn[['frame', 'keypoints']], on='frame', how='left')
    # Replace NaN with list [-1,-1,-1, .  .  . ,-1] (51 item totol)
    allFrame.loc[allFrame['keypoints'].isnull(),['keypoints']] = allFrame.loc[allFrame['keypoints'].isnull(),'keypoints'].apply(lambda keypoints: [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
    # Keypoint
    kpall = allFrame['keypoints']
    # remove c 
    keypointls = list()
    for frm in kpall:
      frm = [round(i,3) for i in frm]  # convert a list into list with 3 decimal places
      for i in frm:
        if i <= 1 and i >= 0 :
            frm.remove(i)
      keypointls.append(frm)

    # convert list to array
    keypointar = np.asarray(keypointls)

    #ปรับค่า X,Y Nose
    arrayper = keypointar
    X_Zero = arrayper[:,0] #X point Zero
    Y_Zero = arrayper[:,1] #Y point Zero
    l=list()
    for i in range(0, 34):
      if(i%2) == 0:
        A = abs(arrayper[:,i] - X_Zero)
      else:
        A = abs(arrayper[:,i] - Y_Zero)
      l.append(A)

    #Z Score
    
    scaler = StandardScaler()
    scaler.fit(l)
    lstand = scaler.transform(l)
    datalistdf = pd.DataFrame(lstand, name_column)
    df = datalistdf.T

    #กรณี มากกว่า 60 เฟรม จะเริ่มเก็บจากเฟรมสุดท้าย
    (naa,nbb) = lstand.shape
    D1 = list()
    for r in range(0,34):
      for t in range(nbb-60,nbb):
        dat1 = lstand[r,t]
        D1.append(dat1)
    datalistdf1 = pd.DataFrame(D1)
    df1 = datalistdf1.T
    df2 = df.values.tolist()

    # กรณีที่จับได้น้อยกว่า 50%
    ccv0 = 0
    for cv1 in df2:
      for cv0 in cv1:
        if cv0 == 0:
          ccv0 = ccv0 + 1
    if ccv0 < 1500:  #old ccv0 < 1020
      featvec.append(df1)

  featvec2 = pd.concat(featvec)
  np.savetxt('app/files/Fature_to_Test.csv', featvec2,delimiter=',' )
  np.savetxt('app/files/save_apic.csv',apic,fmt=['%s','%s', '%f'],delimiter=',')
  data_train(video_input)

def read_json(video_input): #read json file
  os.chdir('/content/drive/My Drive/DetectConcealGun')
  with open('app/files/alphapose-results-forvis-tracked.json','r') as myfile:
    data = myfile.read()
    obj = json.loads(data)
    prepare_idx(obj,video_input)


def  get_poseflow(video_input):
  if os.path.exists('image') :
    shutil.rmtree('image')
  os.mkdir("image")
  !ffmpeg  -i $video_input -r 30 image/%d.jpg    #Extract images

  if os.path.exists('Outimages') : shutil.rmtree('Outimages')
  os.mkdir ("Outimages")
  !pip install -q youtube-dl visdom
  os.chdir( "AlphaPose" )
  !python3 demo.py --sp --indir ../image/ --outdir ../.
  !pip2 install munkres==1.0.12   #poseflow
  os.chdir('/content/drive/My Drive/DetectConcealGun/app/files')
  !python2  AlphaPose/PoseFlow/tracker-general.py --imgdir image/ --in_json alphapose-results.json --out_json alphapose-results-forvis-tracked.json --visdir Outimages/
  os.chdir('/content/drive/My Drive/DetectConcealGun')
  read_json(video_input)

def get_length(filename):
  result = subprocess.run(["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  print('Video length : ' ,float(result.stdout))
  if float(result.stdout) < 2:
    print ("Please upload your video less than 2 second")
  elif float(result.stdout) > 100 :
    print ("Sorry this video is more than 2 second video")
  else :
    get_poseflow(filename)

def main(videoName):
  os.chdir("/content/drive/My Drive/DetectConcealGun/app/files")
  # videoName = str(input("Enter Video name : "))
  get_length(videoName)
  # os.chdir("/content/drive/My Drive/DetectConcealGun")



def allowed_video(filename):
    if not "." in filename:
        return False
    ext = filename.rsplit(".", 1)[1]
    if ext.upper() in app.config["ALLOWED_VIDEO_EXTENSIONS"]:
        return True
    else:
        return False

@app.route("/", methods=["GET", "POST"])
def upload_video():
    if request.method == "POST":
        if request.files:
            video = request.files["video"]

            videoin = 'files/' + video.filename

            if os.path.exists(videoin) :
              print('This is the same video')
              filename = secure_filename(video.filename)
              # read_json(filename)
              filenameout = 'results/result_' +secure_filename(video.filename)
              return render_template("public/result_video.html", videoresult=filenameout)

            if video.filename == "":
                # flash("No file", "warning")
                return redirect(request.url)


            if allowed_video(video.filename):
                filename = secure_filename(video.filename)
                video.save(os.path.join(app.config["VIDEO_UPLOADS"], filename))
                # flash("Video uploaded", "success")


                main(filename)
                # read_json(filename)
                # framestoVDO(filename)
                
                # return redirect(request.url)
                filenameout = 'results/result_' + secure_filename(video.filename)
                return render_template("public/result_video.html", videoresult=filenameout)
                # return redirect(url_for("result_detect",filename=filename))

            else:
                # flash("That file extension is not allowed", "danger")
                return redirect(request.url)
    return render_template("public/upload_video.html")


#PAGE

@app.route("/about")
def about():
    return render_template("public/about.html")
  
@app.route("/result_video")
def result_video():
  return render_template("public/result_video.html")

@app.route("/dataset")
def video_dataset():
  return render_template("public/dataset.html")


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://253e62d6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Mar/2020 02:14:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:14:37] "GET /files/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:14:38] "GET /files/js/app.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:14:39] "GET /files/css/gun32.png HTTP/1.1" 200 -


Video length :  2.047958
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxm

...

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Check Class for Prediction*** 
           0         1         2  ...  0_Predict   Class_0   Class_1
0 -0.989715 -0.980586 -0.987551  ...          0  0.821667  0.178333
1 -0.904263 -0.899140 -0.908894  ...          1  0.083333  0.916667
2 -1.120850 -1.068504 -1.043034  ...          0  0.620000  0.380000

[3 rows x 2043 columns]


127.0.0.1 - - [04/Mar/2020 02:28:26] "POST / HTTP/1.1" 200 -


video created
app/files/results/result_QqBamNKW4KU_dup1_Test402
It's success!


127.0.0.1 - - [04/Mar/2020 02:28:28] "GET /files/css/gun32.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:28:32] "GET /files/results/result_QqBamNKW4KU_dup1_Test402.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [04/Mar/2020 02:28:33] "GET /files/results/result_QqBamNKW4KU_dup1_Test402.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [04/Mar/2020 02:28:34] "GET /files/results/result_QqBamNKW4KU_dup1_Test402.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [04/Mar/2020 02:44:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:47:40] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:47:48] "GET /dataset HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 02:47:49] "GET /files/css/picdataset.jpg HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 03:16:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 06:31:12] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 06:31:14] "GET /dataset HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 06:31:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2020 06:32:22] "GET /about HTTP/1.1" 200 -
1